# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
# Your code here
import requests

import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/isobeldaley/.secret/yelp_api.json")

api_key = keys['api_key']

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
            }


term = 'Pizza'
location = 'NYC'

url = 'https://api.yelp.com/v3/businesses/search'

url_params = {
                'term': term,
                'location': location
    
                }

resp = requests.get(url, headers=headers, params=url_params)

print(resp.text[: 1000])

{"businesses": [{"id": "ysqgdbSrezXgVwER2kQWKA", "alias": "julianas-pizza-brooklyn-5", "name": "Juliana's Pizza", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTxhWHf3YS70Ss_CfxA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/julianas-pizza-brooklyn-5?adjust_creative=nMgr7aeqJTTe8jZ4v4M61w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nMgr7aeqJTTe8jZ4v4M61w", "review_count": 2044, "categories": [{"alias": "pizza", "title": "Pizza"}], "rating": 4.5, "coordinates": {"latitude": 40.7026153030093, "longitude": -73.9934159993549}, "transactions": [], "price": "$$", "location": {"address1": "19 Old Fulton St", "address2": "", "address3": "", "city": "Brooklyn", "zip_code": "11201", "country": "US", "state": "NY", "display_address": ["19 Old Fulton St", "Brooklyn, NY 11201"]}, "phone": "+17185966700", "display_phone": "(718) 596-6700", "distance": 323.20506308227306}, {"id": "zj8Lq1T8KIC5zwFief15jg", "alias": "prince-street-pizza-new-york-2", 

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [2]:
# Your code here; use a function or loop to retrieve all the results from your original request
import pandas as pd
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    resp = requests.get(url, headers=headers, params=url_params)
    
    df = pd.DataFrame(resp.json()['businesses'])
    return df

def all_results(url_params, api_key):
    num = resp.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    dfs = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        dfs.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    df = pd.concat(dfs, ignore_index=True)
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

10800 total matches found.
1000


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,milkflower-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7628961, 'longitude': -73.9207...",(718) 204-1300,461.294783,1OZPOWZwJr-hpvMt6TD4ug,https://s3-media3.fl.yelpcdn.com/bphoto/SQG1kI...,False,"{'address1': '3412 31st Ave', 'address2': '', ...",Milkflower,+17182041300,$$,4.5,600,[],https://www.yelp.com/biz/milkflower-astoria?ad...
1,rizzos-fine-pizza-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.76335, 'longitude': -73.91516}",(718) 721-9862,713.161109,hB2S1y5T9ufMz5ksHHhdIA,https://s3-media3.fl.yelpcdn.com/bphoto/p1IQ1w...,False,"{'address1': '3013 Steinway St', 'address2': '...",Rizzo's Fine Pizza,+17187219862,$,4.0,568,"[delivery, pickup]",https://www.yelp.com/biz/rizzos-fine-pizza-ast...
2,rosas-pizza-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7671877, 'longitude': -73.9299...",(718) 726-1296,655.314626,5Uo_a_Kl53O_yKglSR5nBA,https://s3-media3.fl.yelpcdn.com/bphoto/6VImxk...,False,"{'address1': '31-1 21st St', 'address2': '', '...",Rosa's Pizza,+17187261296,$$,4.5,66,[],https://www.yelp.com/biz/rosas-pizza-astoria?a...
3,mela-pizza-house-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.76765, 'longitude': -73.92056}",(718) 777-5533,159.969828,KjlNK_o3kWtsjsIx6ZgF0A,https://s3-media2.fl.yelpcdn.com/bphoto/2O0JuH...,False,"{'address1': '31-01 Newtown Ave', 'address2': ...",Mela Pizza House,+17187775533,$,4.0,36,"[delivery, pickup]",https://www.yelp.com/biz/mela-pizza-house-asto...
4,basil-brick-oven-pizza-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.770946, 'longitude': -73.92001}",(718) 204-1205,487.240449,JjpCF17AxF6RPOyimw21cw,https://s3-media1.fl.yelpcdn.com/bphoto/VLm8YK...,False,"{'address1': '28-17 Astoria Blvd', 'address2':...",Basil Brick Oven Pizza,+17182041205,$$,4.0,472,"[delivery, pickup]",https://www.yelp.com/biz/basil-brick-oven-pizz...


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [3]:
# Your code here

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [8]:
df['latitude'] = None
df['longitude'] = None

for i in range(0, 1000):
    df['latitude'][i] = df['coordinates'][i]['latitude']
    df['longitude'][i] = df['coordinates'][i]['longitude']

df.head()

/Users/isobeldaley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/isobeldaley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,latitude,longitude
0,milkflower-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7628961, 'longitude': -73.9207...",(718) 204-1300,461.294783,1OZPOWZwJr-hpvMt6TD4ug,https://s3-media3.fl.yelpcdn.com/bphoto/SQG1kI...,False,"{'address1': '3412 31st Ave', 'address2': '', ...",Milkflower,+17182041300,$$,4.5,600,[],https://www.yelp.com/biz/milkflower-astoria?ad...,40.7629,-73.9207
1,rizzos-fine-pizza-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.76335, 'longitude': -73.91516}",(718) 721-9862,713.161109,hB2S1y5T9ufMz5ksHHhdIA,https://s3-media3.fl.yelpcdn.com/bphoto/p1IQ1w...,False,"{'address1': '3013 Steinway St', 'address2': '...",Rizzo's Fine Pizza,+17187219862,$,4.0,568,"[delivery, pickup]",https://www.yelp.com/biz/rizzos-fine-pizza-ast...,40.7634,-73.9152
2,rosas-pizza-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7671877, 'longitude': -73.9299...",(718) 726-1296,655.314626,5Uo_a_Kl53O_yKglSR5nBA,https://s3-media3.fl.yelpcdn.com/bphoto/6VImxk...,False,"{'address1': '31-1 21st St', 'address2': '', '...",Rosa's Pizza,+17187261296,$$,4.5,66,[],https://www.yelp.com/biz/rosas-pizza-astoria?a...,40.7672,-73.93
3,mela-pizza-house-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.76765, 'longitude': -73.92056}",(718) 777-5533,159.969828,KjlNK_o3kWtsjsIx6ZgF0A,https://s3-media2.fl.yelpcdn.com/bphoto/2O0JuH...,False,"{'address1': '31-01 Newtown Ave', 'address2': ...",Mela Pizza House,+17187775533,$,4.0,36,"[delivery, pickup]",https://www.yelp.com/biz/mela-pizza-house-asto...,40.7677,-73.9206
4,basil-brick-oven-pizza-astoria,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.770946, 'longitude': -73.92001}",(718) 204-1205,487.240449,JjpCF17AxF6RPOyimw21cw,https://s3-media1.fl.yelpcdn.com/bphoto/VLm8YK...,False,"{'address1': '28-17 Astoria Blvd', 'address2':...",Basil Brick Oven Pizza,+17182041205,$$,4.0,472,"[delivery, pickup]",https://www.yelp.com/biz/basil-brick-oven-pizz...,40.7709,-73.92


In [12]:
# Your code here

import folium

lat_base = df['latitude'].mean()
long_base = df['longitude'].mean()

base_map = folium.Map([lat_base, long_base], zoom_start=13)

for p in range(0,1000):
    lat = df['latitude'][p]
    long = df['longitude'][p]
    popup_text = 'Latitude: {}, Longitude: {}'.format(lat, long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)

base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!